In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split


# Sample text
text = "Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text. At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model. One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character prediction tasks. Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction plays a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future of text-based technology."""


# Creating character vocabulary
chars = sorted(list(set(text)))
ix_to_char = {i: ch for i, ch in enumerate(chars)}
char_to_ix = {ch: i for i, ch in enumerate(chars)}


# Preparing the dataset for sequence prediction
X = []
y = []
max_length = 10  # Maximum length of input sequences
for i in range(len(text) - max_length - 1):
    sequence = text[i:i + max_length]
    label_sequence = text[i+1:i + max_length + 1]  # Shift by one for the next character sequence
    X.append([char_to_ix[char] for char in sequence])
    y.append([char_to_ix[char] for char in label_sequence])


X = np.array(X)
y = np.array(y)


# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# Converting data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.long)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)


# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)


    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].detach()


# Defining the Transformer model
class CharTransformer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(CharTransformer, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, x):
        embedded = self.embedding(x)
        embedded = self.pos_encoder(embedded)
        transformer_output = self.transformer_encoder(embedded)
        output = self.fc(transformer_output)
        return output


# Hyperparameters
hidden_size = 128
num_layers = 3
nhead = 2
learning_rate = 0.005
epochs = 100


# Model, loss, and optimizer
model = CharTransformer(len(chars), hidden_size, len(chars), num_layers, nhead)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Training the model
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output.transpose(1, 2), y_train)  # Reshape output to match the CrossEntropyLoss expectations
    loss.backward()
    optimizer.step()


    # Validation
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output.transpose(1, 2), y_val)  # Same transpose for validation
        _, predicted = torch.max(val_output, 2)  # Adjust dimension for prediction
        val_accuracy = (predicted == y_val).float().mean()  # Calculate accuracy


    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')


# Prediction function
def predict_next_char(model, char_to_ix, ix_to_char, initial_str):
    model.eval()
    with torch.no_grad():
        initial_input = torch.tensor([char_to_ix[c] for c in initial_str[-max_length:]], dtype=torch.long).unsqueeze(0)
        prediction = model(initial_input)
        predicted_index = torch.argmax(prediction[0, -1], dim=0).item()
        return ix_to_char[predicted_index]


# Predicting the next character
test_str = "Next character prediction is a fundamental task in the field of natural "
predicted_char = predict_next_char(model, char_to_ix, ix_to_char, test_str)
print(f"Predicted next character: '{predicted_char}'")







Epoch 10, Loss: 3.055219888687134, Validation Loss: 3.039682388305664, Validation Accuracy: 0.14694736897945404
Epoch 20, Loss: 3.0347983837127686, Validation Loss: 3.022606372833252, Validation Accuracy: 0.14694736897945404
Epoch 30, Loss: 3.028949737548828, Validation Loss: 3.0128512382507324, Validation Accuracy: 0.14694736897945404
Epoch 40, Loss: 3.0252997875213623, Validation Loss: 3.0100173950195312, Validation Accuracy: 0.14694736897945404
Epoch 50, Loss: 3.0255377292633057, Validation Loss: 3.009488582611084, Validation Accuracy: 0.14694736897945404
Epoch 60, Loss: 3.024329662322998, Validation Loss: 3.0099596977233887, Validation Accuracy: 0.14694736897945404
Epoch 70, Loss: 3.022263526916504, Validation Loss: 3.009981155395508, Validation Accuracy: 0.14694736897945404
Epoch 80, Loss: 3.021327257156372, Validation Loss: 3.0098352432250977, Validation Accuracy: 0.14694736897945404
Epoch 90, Loss: 3.0210843086242676, Validation Loss: 3.009847402572632, Validation Accuracy: 0.14

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import time
from sklearn.model_selection import train_test_split


# Sample text
text = """Next character prediction is a fundamental task in the field of natural language processing (NLP) that involves predicting the next character in a sequence of text based on the characters that precede it. This task is essential for various applications, including text auto-completion, spell checking, and even in the development of sophisticated AI models capable of generating human-like text. At its core, next character prediction relies on statistical models or deep learning algorithms to analyze a given sequence of text and predict which character is most likely to follow. These predictions are based on patterns and relationships learned from large datasets of text during the training phase of the model. One of the most popular approaches to next character prediction involves the use of Recurrent Neural Networks (RNNs), and more specifically, a variant called Long Short-Term Memory (LSTM) networks. RNNs are particularly well-suited for sequential data like text, as they can maintain information in 'memory' about previous characters to inform the prediction of the next character. LSTM networks enhance this capability by being able to remember long-term dependencies, making them even more effective for next character prediction tasks. Training a model for next character prediction involves feeding it large amounts of text data, allowing it to learn the probability of each character's appearance following a sequence of characters. During this training process, the model adjusts its parameters to minimize the difference between its predictions and the actual outcomes, thus improving its predictive accuracy over time. Once trained, the model can be used to predict the next character in a given piece of text by considering the sequence of characters that precede it. This can enhance user experience in text editing software, improve efficiency in coding environments with auto-completion features, and enable more natural interactions with AI-based chatbots and virtual assistants. In summary, next character prediction plays a crucial role in enhancing the capabilities of various NLP applications, making text-based interactions more efficient, accurate, and human-like. Through the use of advanced machine learning models like RNNs and LSTMs, next character prediction continues to evolve, opening new possibilities for the future of text-based technology."""


# Creating character vocabulary
chars = sorted(list(set(text)))
ix_to_char = {i: ch for i, ch in enumerate(chars)}
char_to_ix = {ch: i for i, ch in enumerate(chars)}


# Function to prepare data for training and validation with different sequence lengths
def prepare_data(seq_length):
    X = []
    y = []
    for i in range(len(text) - seq_length - 1):
        sequence = text[i:i + seq_length]
        label_sequence = text[i + 1:i + seq_length + 1]  # Shift by one for the next character sequence
        X.append([char_to_ix[char] for char in sequence])
        y.append([char_to_ix[char] for char in label_sequence])


    X = np.array(X)
    y = np.array(y)
    
    return X, y


# Set sequence lengths
sequence_lengths = [10, 20, 30]
results = {}


# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)


    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].detach()


# Defining the Encoder (self-attention) and Decoder (cross-attention) for the Transformer model
class CharTransformerWithCrossAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, nhead):
        super(CharTransformerWithCrossAttention, self).__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.pos_encoder = PositionalEncoding(hidden_size)
        
        # Encoder layer (self-attention)
        encoder_layers = nn.TransformerEncoderLayer(hidden_size, nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        
        # Decoder layer (cross-attention)
        decoder_layers = nn.TransformerDecoderLayer(hidden_size, nhead, batch_first=True)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layers, num_layers)
        
        self.fc = nn.Linear(hidden_size, output_size)


    def forward(self, x, memory=None):
        embedded = self.embedding(x)  # [batch_size, seq_len, hidden_size]
        embedded = self.pos_encoder(embedded)
        
        encoder_output = self.transformer_encoder(embedded)
        
        if memory is not None:
            decoder_output = self.transformer_decoder(embedded, memory)  # Use cross-attention
        else:
            decoder_output = self.transformer_decoder(embedded, encoder_output)  # Use self-attention
        
        output = self.fc(decoder_output)  # [batch_size, seq_len, output_size]
        return output


# Training and evaluation loop
def train_and_evaluate(sequence_length):
    X, y = prepare_data(sequence_length)


    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.long)
    y_train = torch.tensor(y_train, dtype=torch.long)
    X_val = torch.tensor(X_val, dtype=torch.long)
    y_val = torch.tensor(y_val, dtype=torch.long)


    # Hyperparameters
    hidden_size = 128
    num_layers = 3
    nhead = 2
    learning_rate = 0.005
    epochs = 100


    # Model, loss, and optimizer
    model = CharTransformerWithCrossAttention(len(chars), hidden_size, len(chars), num_layers, nhead)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)


    start_time = time.time()


    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output.transpose(1, 2), y_train)
        loss.backward()
        optimizer.step()


        # Validation
        model.eval()
        with torch.no_grad():
            val_output = model(X_val)
            val_loss = criterion(val_output.transpose(1, 2), y_val)
            _, predicted = torch.max(val_output, 2)
            val_accuracy = (predicted == y_val).float().mean()


        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}, Loss: {loss.item()}, Validation Loss: {val_loss.item()}, Validation Accuracy: {val_accuracy.item()}')


    execution_time = time.time() - start_time


    model_size = sum(p.numel() for p in model.parameters() if p.requires_grad)


    return execution_time, model_size, val_accuracy.item()


# Run training and evaluation for different sequence lengths
for seq_len in sequence_lengths:
    print(f"Training with sequence length: {seq_len}")
    exec_time, model_size, val_accuracy = train_and_evaluate(seq_len)
    results[seq_len] = {
        'Execution Time': exec_time,
        'Model Size (params)': model_size,
        'Validation Accuracy': val_accuracy
    }


# Print the results for different sequence lengths
for seq_len, result in results.items():
    print(f"\nResults for sequence length {seq_len}:")
    print(f"Execution Time: {result['Execution Time']} seconds")
    print(f"Model Size: {result['Model Size (params)']} parameters")
    print(f"Validation Accuracy: {result['Validation Accuracy']}")







Training with sequence length: 10
Epoch 10, Loss: 3.045806646347046, Validation Loss: 3.0274105072021484, Validation Accuracy: 0.14694736897945404
Epoch 20, Loss: 3.0340068340301514, Validation Loss: 3.01540470123291, Validation Accuracy: 0.14694736897945404
Epoch 30, Loss: 3.02724289894104, Validation Loss: 3.0120341777801514, Validation Accuracy: 0.14694736897945404
Epoch 40, Loss: 3.0253703594207764, Validation Loss: 3.011526584625244, Validation Accuracy: 0.14694736897945404
Epoch 50, Loss: 3.02331280708313, Validation Loss: 3.0111489295959473, Validation Accuracy: 0.14694736897945404
Epoch 60, Loss: 3.021718978881836, Validation Loss: 3.0104732513427734, Validation Accuracy: 0.14694736897945404
Epoch 70, Loss: 3.0228395462036133, Validation Loss: 3.0100510120391846, Validation Accuracy: 0.14694736897945404
Epoch 80, Loss: 3.0216000080108643, Validation Loss: 3.009760856628418, Validation Accuracy: 0.14694736897945404
Epoch 90, Loss: 3.0205280780792236, Validation Loss: 3.009863615